In [ ]:
### Codeblocks that can be added to Main.ipynb after the model has been trained to create misc visualizations and to analysis the data.

In [ ]:
### Evaluation Area ###
# Visualize the concepts

index = 444
image, true_label, true_concepts = derm7pt[index]
image = image.to(device)

# Get model predictions
with torch.no_grad():
    # to Note! this is the model from the last fold in the for loop
    predicted_concepts, predicted_label = model(image)

predicted_concepts = predicted_concepts.squeeze(0)
predicted_concepts = predicted_concepts.to('cpu')
image = image.to('cpu')


# Plot
plt.rcParams.update({'font.size' : 14})
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
concept_names = [name.replace('_num', '') for name in Derm7pt_data.model_columns['concepts']]
concept_names = [name.replace('_', ' ') for name in concept_names]
# image
ax[0].imshow(image.squeeze().permute(1, 2, 0))
ax[0].set_title("Input Image")
ax[0].axis('off')
# True Concepts
ax[1].barh(concept_names, true_concepts, color='mediumseagreen')
ax[1].set_title("True Concepts")
ax[1].set_xlim(0, 1)
ax[1].set_xlabel('Concept Value')
# Predicted Concepts
ax[2].barh(concept_names, predicted_concepts, color='dodgerblue')
ax[2].set_title("Predicted Concepts")
ax[2].set_xlim(0, 1)
ax[2].set_xlabel('Concept Value')
# Adjust layout to make space for titles
plt.tight_layout()

# Save the image
output_folder = 'output_Images'
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, f'concept_image_{index}.png')
plt.savefig(output_path)


plt.show()
print(predicted_label)


In [1]:
# Visualization of concepts, but True and predicted concepts are shown in same plot
import numpy as np
# Plot
plt.rcParams.update({'font.size' : 16})
fig, ax = plt.subplots(1, 2, figsize=(15, 7))

# Concept names
concept_names = [name.replace('_num', '') for name in Derm7pt_data.model_columns['concepts']]
concept_names = [name.replace('_', ' ') for name in concept_names]
y_pos = np.arange(len(concept_names))

# Image
ax[0].imshow(image.squeeze().permute(1, 2, 0))
ax[0].set_title("Input Image")
ax[0].axis('off')

# Grouped bar chart for true and predicted concepts
bar_width = 0.35  # Width of the bars

# Plot true and predicted values side by side
ax[1].barh(y_pos - bar_width/2, true_concepts, bar_width, label='True', color='mediumseagreen')
ax[1].barh(y_pos + bar_width/2, predicted_concepts, bar_width, label='Predicted', color='dodgerblue')

# Set the chart's labels and title
ax[1].set_yticks(y_pos)
ax[1].set_yticklabels(concept_names)
ax[1].set_title("True vs Predicted Concepts")
ax[1].set_xlim(0, 1)  # Set limit for the x-axis (concept values)
ax[1].set_xlabel('Concept Value')
ax[1].legend()

# Adjust layout to make space for titles
plt.tight_layout()

# Save the image
output_folder = 'output_Images'
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, f'concept_image_grouped_{index}.png')
plt.savefig(output_path)

plt.show()


In [ ]:
# Find examples for visualization
def find_image_with_most_ones(indices, mode="val"):
    max_ones = 0
    max_ones_index = -1

    for idx in indices:
        _, _, concepts = derm7pt[idx]
        num_ones = torch.sum(concepts).item()  # Count the number of 1s in the concepts

        if num_ones > max_ones:
            max_ones = num_ones
            max_ones_index = idx

        if num_ones > 2:
            print(f"Found an image with {num_ones} ones in concepts in {mode} set: {idx}")

    return max_ones_index

# Find the image with the most 1s in the concepts for both train and val sets
train_max_ones_index = find_image_with_most_ones(train_idx, "train")
val_max_ones_index = find_image_with_most_ones(val_idx)

# Print the results
print(f"Train set image index with the most 1s in concepts: {train_max_ones_index}")
print(f"Validation set image index with the most 1s in concepts: {val_max_ones_index}")


In [ ]:
# analyze connection of concept layer and label layer
i = 0
for layer in model.children():
    i = i+1
    if i == 10:
        print("layer", i)
        weight = layer.state_dict()['weight']
        bias = layer.state_dict()['bias']
        print("weight: ", weight)
        print("bias: ", bias)

input_tensor = predicted_concepts
print("input_tensor: ", input_tensor)
input_tensor = input_tensor.to(device)
output = torch.matmul(input_tensor, weight.T) + bias
print(torch.softmax(output, 0))
print("true label: ", derm7pt[89][1])

In [2]:
# unfinshed visualization of concept -> label layer
import matplotlib.pyplot as plt
import numpy as np

# Inputs from the second-to-last layer (7 inputs)
input_tensor = np.array([1.4559e-01, 2.9092e-01, 2.6569e-03, 9.9728e-01, 9.8169e-01, 3.3091e-04, 3.8738e-05])

# Weights for the final linear layer (for label 1)
weights_label_1 = np.array([0.6878, 0.7810, 0.4819, 0.8467, -0.1470, 0.3153, 0.2474])

# Bias for label 1
bias_label_1 = 0.5086

# Calculate the weighted contributions for label 1
contributions_label_1 = input_tensor * weights_label_1

# Sum of weighted contributions plus bias
output_label_1 = np.sum(contributions_label_1) + bias_label_1

# Visualization setup
fig, ax = plt.subplots(figsize=(8, 6))

# Plot input nodes as circles
for i, input_value in enumerate(input_tensor):
    ax.plot(0, i, 'go', markersize=10)  # Input nodes
    ax.text(-0.2, i, f"Input {i+1}\n{input_value:.3f}", va='center', ha='center', fontsize=10, color='green')

# Plot the output node
ax.plot(2, 3, 'bo', markersize=15)  # Output node
ax.text(2.2, 3, f"Output\n{output_label_1:.3f}", va='center', ha='left', fontsize=12, color='blue')

# Draw arrows for each input to output with the weight labels
for i, (input_value, weight, contribution) in enumerate(zip(input_tensor, weights_label_1, contributions_label_1)):
    ax.arrow(0.1, i, 1.6, 3 - i, head_width=0.1, head_length=0.1, fc='gray', ec='gray')
    ax.text(1, (i + 3) / 2, f"{weight:.3f}", ha='center', fontsize=10, color='black')

# Add the bias as an arrow pointing directly to the output
ax.arrow(1.2, 3.5, 0.8, -0.5, head_width=0.1, head_length=0.1, fc='blue', ec='blue')
ax.text(1.8, 3.7, f"Bias\n{bias_label_1:.3f}", ha='center', fontsize=10, color='blue')

# Styling the plot
ax.set_xlim(-1, 3)
ax.set_ylim(-1, 7)
ax.axis('off')  #
